In [2]:
import openrouteservice
from openrouteservice import convert
import folium
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [3]:
traffic_data=(r'..\data\traffic_data.xlsx')
satelit_data=(r'..\data\AgERA5_4params_graz.csv')
temporal_data=(r'..\data\new_5sta(DNOWS)_3com(NO2,PM10K,O3)_year2010_TemporalDummies_Ox_holidays_meteo.csv')

In [4]:
df_satelit= pd.read_csv(satelit_data,index_col = 0)
df_temporal =pd.read_csv(temporal_data,index_col = 0)

In [5]:
df_satelit.index = pd.to_datetime(df_satelit.index)

In [6]:
df_temporal.index = pd.to_datetime(df_temporal.index)

In [7]:
df_ulice = pd.read_excel(traffic_data, engine='openpyxl', sheet_name='street', index_col=0)
df_ulice_podaci = pd.read_excel(traffic_data, engine='openpyxl', sheet_name='traffic_data', index_col=0)

In [8]:
df_ulice_podaci=df_ulice_podaci['2018-01-01':'2020-03-15']
df_satelit=df_satelit['2018-01-01':'2020-03-15']
df_temporal=df_temporal['2018-01-01':'2020-03-15']

In [9]:
df_ulice = df_ulice.drop(['Unnamed: 1','Street','Abschnitt',],axis = 1)

In [10]:
df_ulice = df_ulice.drop(['Nord','Sud','West','Ost','DonBosco'],axis = 0)

In [11]:
df_ulice_podaci = df_ulice_podaci.fillna(df_ulice_podaci.mean())

In [12]:
df_temporal_new = df_temporal[['year', 'dayofyear',
       'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan',
       'month_Jul', 'month_Jun', 'month_Mar', 'month_May', 'month_Nov',
       'month_Oct', 'month_Sep', 'weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday', 'season_fall', 'season_spring',
       'season_summer', 'season_winter','holiday',
       'holiday_school']]
df_satelit_new = df_satelit[['Cloud_Cover_Mean','Wind_Speed_10m_Mean']]

In [13]:
client = openrouteservice.Client(key='5b3ce3597851110001cf624866bbf1eb192e45148c3b15f3088db4c2')

In [14]:
df_ulice

,Directions,Longitude,Latitude,Reference_station,Distance to ref,AVG traffic,num_of_roads,semafor,Unnamed: 12
Code,,,,,,,,,
Ost_8010_208,SouthNorth,15.443536,47.077866,Ost,2689.680221,963.134172,18.0,1.0,NaN
DonBosco_8020_110,SouthNorth,15.418848,47.058659,DonBosco,381.001390,985.662456,13.0,1.0,bus stop near by
DonBosco_8020_119,EastWest,15.431343,47.066870,DonBosco,1682.330202,455.922269,7.0,1.0,NaN
DonBosco_8020_122,EastWest,15.431727,47.057316,DonBosco,1169.351420,1214.595238,15.0,1.0,middle of city
DonBosco_8053_111,SouthNorth,15.414219,47.043459,DonBosco,1363.162404,914.792017,13.0,1.0,NaN
Nord_8010_205,EastWest,15.426301,47.087794,Nord,1120.294237,645.190077,4.0,0.0,tunnel
Ost_8010_209,EastWest,15.457930,47.076679,Ost,2017.995362,602.967855,12.0,1.0,dosta drveca mozda dio NO2 adsorbiraju
Ost_8010_213,East,15.444325,47.065714,Ost,1828.992486,595.693978,5.0,1.0,NaN
Ost_8010_214,SouthNorth,15.448640,47.054039,Ost,1497.288645,696.783844,9.0,1.0,NaN


In [15]:
df_ulice=df_ulice.drop('Sud_8020_107',axis=0)

In [16]:
m = folium.Map(location=[47.0707, 15.4395],zoom_start=10, control_scale=True,tiles="cartodbpositron")
dictionary = {'postaja':[],'udaljenost_u_km':[]}
for longitude,latitude,postaja, in zip(df_ulice.Longitude,df_ulice.Latitude,df_ulice.index):
    coords = ((longitude,latitude),(15.429734,47.045648))  
    res = client.directions(coords)
    geometry = client.directions(coords)['routes'][0]['geometry']
    decoded = convert.decode_polyline(geometry)
    distance_txt = "<h4> <b>Distance :&nbsp" + "<strong>"+str(round(res['routes'][0]['summary']['distance']/1000,1))+" Km </strong>" +"</h4></b>"
    duration_txt = "<h4> <b>Duration :&nbsp" + "<strong>"+str(round(res['routes'][0]['summary']['duration']/60,1))+" Mins. </strong>" +"</h4></b>"
    udaljenost = round(res['routes'][0]['summary']['distance']/1000,1)
    dictionary['postaja'].append(postaja)
    dictionary['udaljenost_u_km'].append(udaljenost)
 
    folium.GeoJson(decoded).add_child(folium.Popup(distance_txt+duration_txt,max_width=300)).add_to(m)
    folium.Marker(
    location=list(coords[0][::-1]),
    popup=postaja,
    icon=folium.Icon(color="green"),
        ).add_to(m)

    folium.Marker(
    location=list(coords[1][::-1]),
    popup='Sud_8020_107',
    icon=folium.Icon(color="red"),
        ).add_to(m)


In [17]:
m

In [18]:
data = pd.DataFrame.from_dict(dictionary)
data

,postaja,udaljenost_u_km
0,Ost_8010_208,4.8
1,DonBosco_8020_110,2.2
2,DonBosco_8020_119,2.9
3,DonBosco_8020_122,1.5
4,DonBosco_8053_111,2.2
5,Nord_8010_205,5.4
6,Ost_8010_209,5.1
7,Ost_8010_213,4.1
8,Ost_8010_214,3.2
9,Nord_8020_102,5.2


In [19]:
data['norm']=data['udaljenost_u_km']/data['udaljenost_u_km'].max()
data

,postaja,udaljenost_u_km,norm
0,Ost_8010_208,4.8,0.657534
1,DonBosco_8020_110,2.2,0.301370
2,DonBosco_8020_119,2.9,0.397260
3,DonBosco_8020_122,1.5,0.205479
4,DonBosco_8053_111,2.2,0.301370
5,Nord_8010_205,5.4,0.739726
6,Ost_8010_209,5.1,0.698630
7,Ost_8010_213,4.1,0.561644
8,Ost_8010_214,3.2,0.438356
9,Nord_8020_102,5.2,0.712329


In [20]:
X = df_ulice_podaci.drop('Sud_8020_107',axis=1)
for i,j in zip(X.columns,data.norm):
    X[i]=X[i]*j

In [21]:
Y = df_ulice_podaci[['DonBosco_8020_119']]
Y

,DonBosco_8020_119
2018-01-01,624.0
2018-01-02,772.0
2018-01-03,772.0
2018-01-04,801.0
2018-01-05,852.0
...,...
2020-03-11,469.0
2020-03-12,478.0
2020-03-13,492.0
2020-03-14,341.0


In [22]:
models = {'lr':LinearRegression,
          'kn':KNeighborsRegressor,
          'rf':RandomForestRegressor,
          'dt':DecisionTreeRegressor}

# Udaljenost i traffic i ostali podaci 



In [23]:
Xf=df_ulice_podaci.drop('Sud_8020_107',axis=1)

In [30]:
data = pd.concat([Xf,X,df_satelit_new,df_temporal_new],axis=1)

In [32]:
data.columns

Index(['Ost_8010_208', 'DonBosco_8020_110', 'DonBosco_8020_119',
       'DonBosco_8020_122', 'DonBosco_8053_111', 'Nord_8010_205',
       'Ost_8010_209', 'Ost_8010_213', 'Ost_8010_214', 'Nord_8020_102',
       'West_8020_103', 'Sud_8041_218', 'Ost_8041_221', 'Nord_8045_203',
       'Nord_8051_101', 'Ost_8010_208', 'DonBosco_8020_110',
       'DonBosco_8020_119', 'DonBosco_8020_122', 'DonBosco_8053_111',
       'Nord_8010_205', 'Ost_8010_209', 'Ost_8010_213', 'Ost_8010_214',
       'Nord_8020_102', 'West_8020_103', 'Sud_8041_218', 'Ost_8041_221',
       'Nord_8045_203', 'Nord_8051_101', 'Cloud_Cover_Mean',
       'Wind_Speed_10m_Mean', 'year', 'dayofyear', 'month_Apr', 'month_Aug',
       'month_Dec', 'month_Feb', 'month_Jan', 'month_Jul', 'month_Jun',
       'month_Mar', 'month_May', 'month_Nov', 'month_Oct', 'month_Sep',
       'weekday_Friday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'season_fal

In [33]:
data

,Ost_8010_208,DonBosco_8020_110,DonBosco_8020_119,DonBosco_8020_122,DonBosco_8053_111,Nord_8010_205,Ost_8010_209,Ost_8010_213,Ost_8010_214,Nord_8020_102,...,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,season_fall,season_spring,season_summer,season_winter,holiday,holiday_school
2018-01-01,663.0,534.0,624.0,668.0,467.0,352.0,344.0,364.0,341.0,622.0,...,0,0,0,0,0,0,0,1,1,1
2018-01-02,927.0,924.0,772.0,1191.0,904.0,673.0,563.0,544.0,702.0,1181.0,...,0,0,1,0,0,0,0,1,0,1
2018-01-03,962.0,922.0,772.0,1206.0,883.0,699.0,587.0,553.0,711.0,1149.0,...,0,0,0,1,0,0,0,1,0,1
2018-01-04,978.0,941.0,801.0,1233.0,900.0,720.0,606.0,565.0,726.0,1176.0,...,0,1,0,0,0,0,0,1,0,1
2018-01-05,1027.0,986.0,852.0,1278.0,946.0,743.0,613.0,604.0,762.0,1278.0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-11,931.0,1193.0,469.0,1342.0,1019.0,683.0,698.0,680.0,812.0,1396.0,...,0,0,0,1,0,1,0,0,0,0
2020-03-12,922.0,1158.0,478.0,1333.0,992.0,652.0,664.0,669.0,792.0,1366.0,...,0,1,0,0,0,1,0,0,0,0
2020-03-13,892.0,1166.0,492.0,1371.0,989.0,655.0,639.0,689.0,817.0,1424.0,...,0,0,0,0,0,1,0,0,0,0
2020-03-14,487.0,727.0,341.0,877.0,613.0,320.0,359.0,382.0,447.0,872.0,...,0,0,0,0,0,1,0,0,0,0


In [25]:
dictionary={}
temp_dict={}
for model_key in models.keys():
        model= models[model_key]
        model = model()
        
        X_train,X_test,Y_train,Y_test = train_test_split(data,Y, random_state = 0)
        model.fit(X_train,Y_train)
        predictions = model.predict(X_test)
        r2 = r2_score(Y_test, predictions)
        
        temp_dict.update({model_key:r2})
dictionary.update({'Sud_8020_107':temp_dict}) 

C:\Users\vpetric\AppData\Local\Temp/ipykernel_16884/829984425.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,Y_train)


In [26]:
data = pd.DataFrame.from_dict(dictionary)

In [27]:
data

,Sud_8020_107
dt,0.998667
kn,0.911747
lr,1.000000
rf,0.997489


In [28]:
data = pd.concat([X,df_satelit_new,df_temporal_new],axis=1)
data

,Ost_8010_208,DonBosco_8020_110,DonBosco_8020_119,DonBosco_8020_122,DonBosco_8053_111,Nord_8010_205,Ost_8010_209,Ost_8010_213,Ost_8010_214,Nord_8020_102,...,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,season_fall,season_spring,season_summer,season_winter,holiday,holiday_school
2018-01-01,435.945205,160.931507,247.890411,137.260274,140.739726,260.383562,240.328767,204.438356,149.479452,443.068493,...,0,0,0,0,0,0,0,1,1,1
2018-01-02,609.534247,278.465753,306.684932,244.726027,272.438356,497.835616,393.328767,305.534247,307.726027,841.260274,...,0,0,1,0,0,0,0,1,0,1
2018-01-03,632.547945,277.863014,306.684932,247.808219,266.109589,517.068493,410.095890,310.589041,311.671233,818.465753,...,0,0,0,1,0,0,0,1,0,1
2018-01-04,643.068493,283.589041,318.205479,253.356164,271.232877,532.602740,423.369863,317.328767,318.246575,837.698630,...,0,1,0,0,0,0,0,1,0,1
2018-01-05,675.287671,297.150685,338.465753,262.602740,285.095890,549.616438,428.260274,339.232877,334.027397,910.356164,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-11,612.164384,359.534247,186.315068,275.753425,307.095890,505.232877,487.643836,381.917808,355.945205,994.410959,...,0,0,0,1,0,1,0,0,0,0
2020-03-12,606.246575,348.986301,189.890411,273.904110,298.958904,482.301370,463.890411,375.739726,347.178082,973.041096,...,0,1,0,0,0,1,0,0,0,0
2020-03-13,586.520548,351.397260,195.452055,281.712329,298.054795,484.520548,446.424658,386.972603,358.136986,1014.356164,...,0,0,0,0,0,1,0,0,0,0
2020-03-14,320.219178,219.095890,135.465753,180.205479,184.739726,236.712329,250.808219,214.547945,195.945205,621.150685,...,0,0,0,0,0,1,0,0,0,0


In [29]:
data.columns

Index(['Ost_8010_208', 'DonBosco_8020_110', 'DonBosco_8020_119',
       'DonBosco_8020_122', 'DonBosco_8053_111', 'Nord_8010_205',
       'Ost_8010_209', 'Ost_8010_213', 'Ost_8010_214', 'Nord_8020_102',
       'West_8020_103', 'Sud_8041_218', 'Ost_8041_221', 'Nord_8045_203',
       'Nord_8051_101', 'Cloud_Cover_Mean', 'Wind_Speed_10m_Mean', 'year',
       'dayofyear', 'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb',
       'month_Jan', 'month_Jul', 'month_Jun', 'month_Mar', 'month_May',
       'month_Nov', 'month_Oct', 'month_Sep', 'weekday_Friday',
       'weekday_Monday', 'weekday_Saturday', 'weekday_Sunday',
       'weekday_Thursday', 'weekday_Tuesday', 'weekday_Wednesday',
       'season_fall', 'season_spring', 'season_summer', 'season_winter',
       'holiday', 'holiday_school'],
      dtype='object')

In [52]:
dictionary={}
temp_dict={}
for model_key in models.keys():
        model= models[model_key]
        model = model()
        
        X_train,X_test,Y_train,Y_test = train_test_split(data,Y, random_state = 0)
        model.fit(X_train,Y_train)
        predictions = model.predict(X_test)
        r2 = r2_score(Y_test, predictions)
        
        temp_dict.update({model_key:r2})
dictionary.update({'Sud_8020_107':temp_dict}) 

C:\Users\vpetric\AppData\Local\Temp/ipykernel_14136/829984425.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,Y_train)


In [53]:
data = pd.DataFrame.from_dict(dictionary)
data

,Sud_8020_107
dt,0.997728
kn,0.864733
lr,1.000000
rf,0.997754


In [1]:
data

NameError: name 'data' is not defined